In [1]:
!pip install pandas
!pip install requests
!pip install bs4
!pip install plotly
!pip install numpy
!pip install geopy
!pip install html5lib
!pip install sqlalchemy==1.3.9
!pip install ipython-sql
!pip install psycopg2-binary

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import html5lib
import psycopg2

In [3]:
html_data = requests.get("https://www.otomoto.pl/oferta/bmw-m5-600km-m5-individual-kolor-carbon-pakiet-ceramiczne-hamulce-salon-pl-ID6DQdad.html")
soup = BeautifulSoup(html_data.content, 'html5lib')

In [4]:
eur = 4.57
print(eur)


4.57


In [5]:
def get_price(soup):
    price = soup.find('span',attrs={"class":"offer-price__number"}).text
    price = price.replace(" ","")
    if "PLN" in price:
        price = price.replace("PLN","")
        price = float(price)
    elif "EUR" in price:
        price = price.replace("EUR","")
        price = float(price)
        price = price * eur
    else:
        price = None
    return price

In [6]:
def take_id(soup):
    ID = soup.find('div',attrs={"class":"offer-content__metabar"})
    ID = ID.find('div')
    ID = ID.find('span',attrs={"id":"ad_id"}).text
    return ID

In [7]:
def take_details(soup):
    details = {}    
    for info in soup.find_all('li',attrs={"class":"offer-params__item"}):
        key = info.find('span').text.strip()
        value = info.find('div').text.strip()
        details[key] = value
    return details

In [8]:
def take_all(soup):
    dict_all = {}
    dict_all["ID"] = take_id(soup)
    dict_all["price"] = get_price(soup)
    details = take_details(soup)
    return dict_all | details

In [9]:
def data_clean(details):
    try:
        details['Przebieg'] = details['Przebieg'].replace(" km","").replace(" ","")
    except:
        pass
    try:
        details['Moc'] = details['Moc'].replace(" KM","")
    except:
        pass
    try:
        details['Pojemność skokowa'] = details['Pojemność skokowa'].replace(" cm3","").replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Spalanie W Mieście'] = details['Spalanie W Mieście	'].replace("l/100km","").replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Spalanie W Cyklu Mieszanym'] = details['Spalanie W Cyklu Mieszanym'].replace("l/100km","").replace(" ","").replace(",",".")
    except:
        pass
    try:
        details['Spalanie Poza Miastem'] = details['Spalanie Poza Miastem'].replace("l/100km","").replace(" ","").replace(",",".")
    except:
        pass
    return details

In [10]:
car_brand = [
  "Abarth",
  "Alfa Romeo",
  "Aston Martin",
  "Audi",
  "Bentley",
  "BMW",
  "Bugatti",
  "Cadillac",
  "Chevrolet",
  "Chrysler",
  "Citroën",
  "Dacia",
  "Daewoo",
  "Daihatsu",
  "Dodge",
  "Donkervoort",
  "DS",
  "Ferrari",
  "Fiat",
  "Fisker",
  "Ford",
  "Honda",
  "Hummer",
  "Hyundai",
  "Infiniti",
  "Iveco",
  "Jaguar",
  "Jeep",
  "Kia",
  "KTM",
  "Lada",
  "Lamborghini",
  "Lancia",
  "Land Rover",
  "Landwind",
  "Lexus",
  "Lotus",
  "Maserati",
  "Maybach",
  "Mazda",
  "McLaren",
  "Mercedes-Benz",
  "MG",
  "Mini",
  "Mitsubishi",
  "Morgan",
  "Nissan",
  "Opel",
  "Peugeot",
  "Porsche",
  "Renault",
  "Rolls-Royce",
  "Rover",
  "Saab",
  "Seat",
  "Skoda",
  "Smart",
  "SsangYong",
  "Subaru",
  "Suzuki",
  "Tesla",
  "Toyota",
  "Volkswagen",
  "Volvo"
]

In [ ]:
Link_ERROR = 0
Read_announcements = 0
new_row = {}
all_data = []
for brand in car_brand:
    brand_link = f"https://www.otomoto.pl/osobowe/{brand}/?search%5Border%5D=created_at%3Adesc&page="
    for page_main in range(1,3): 
        try:
            html_data_link = requests.get(brand_link + str(page_main))
            soup_url = BeautifulSoup(html_data_link.content, 'html5lib')
        except Exception:
            print("Link_ERROR")
            continue
        links = []
        for offer in soup_url.find_all('a',attrs={"class":"offer-title__link"}):
            if "https://www.otomoto.pl" in offer['href']:
                links.append(offer['href']) 
        for page in links:
            html_data = requests.get(page)
            soup = BeautifulSoup(html_data.content, 'html5lib')
            dict_all = take_all(soup)
            dict_all = data_clean(dict_all)
            all_data.append(dict_all)
            Read_announcements += 1
            clear_output(wait=True)
            print("Read_announcements ", Read_announcements)
table_cars = pd.DataFrame.from_dict(all_data, orient='columns')
delete_duplicates = Read_announcements - len(table_cars.index) 
print("Delete duplicates: ", delete_duplicates)
print("Finish")

Read_announcements  1320


In [ ]:
page_link = brand_link + str(page_main)
print(page_link)

In [ ]:
print(brand_link,page_main)

In [ ]:
#GIT
# Link_ERROR = 0
# Read_announcements = 0
# new_row = {}
# all_data = []
# for page_main in range(1,2):  
#     try:
#         html_data_link = requests.get(f"https://www.otomoto.pl/osobowe/?search%5Border%5D=created_at%3Adesc&page={page_main}")
#         soup_url = BeautifulSoup(html_data_link.content, 'html5lib')
#     except Exception:
#         print("Link_ERROR")
#         continue
#     links = []
#     for offer in soup_url.find_all('a',attrs={"class":"offer-title__link"}):
#         if "https://www.otomoto.pl" in offer['href']:
#             links.append(offer['href']) 
#     for page in links:
#         html_data = requests.get(page)
#         soup = BeautifulSoup(html_data.content, 'html5lib')
#         dict_all = take_all(soup)
#         dict_all = data_clean(dict_all)
#         all_data.append(dict_all)
#         Read_announcements += 1
#         clear_output(wait=True)
#         print("Read_announcements ", Read_announcements)
# table_cars = pd.DataFrame.from_dict(all_data, orient='columns')
# delete_duplicates = Read_announcements - len(table_cars.index) 
# print("Delete duplicates: ", delete_duplicates)
# print("Finish")

In [ ]:
table_cars.head()

In [ ]:
table_name = 'Otomoto'

In [ ]:
table_cars.to_csv (table_name +'.csv', index = False, header=True)

In [ ]:
psql = psycopg2.connect(host='192.168.10.163', port='5432', database='Otomoto', user='barto', password='biznes')
cur = psql.cursor()

In [ ]:
cur.execute('DROP TABLE IF EXISTS '+ table_name)
createTableCommand = 'CREATE TABLE ' + table_name + '''(ID int NOT NULL,
    Oferta od VARCHAR,
    Kategoria VARCHAR,
    Marka pojazdu VARCHAR,
    Model pojazdu VARCHAR,
    Wersja VARCHAR,
    Generacja VARCHAR,
    Rok produkcji INT,
    Przebieg,
    Pojemność skokowa,
    Rodzaj paliwa VARCHAR,
    Moc,
    Skrzynia biegów VARCHAR,
    Napęd VARCHAR,
    Typ nadwozia VARCHAR,
    Emisja CO2,
    Liczba drzwi,
    Liczba miejsc,
    Kolor VARCHAR,
    Rodzaj koloru VARCHAR,
    VAT marża VARCHAR,
    Zarejestrowany w Polsce VARCHAR,
    Pierwszy właściciel VARCHAR,
    Bezwypadkowy VARCHAR,
    Stan VARCHAR,
    Kraj pochodzenia VARCHAR,
    Pierwsza rejestracja,
    Serwisowany w ASO,
    Możliwość finansowania,
    Faktura VAT,
    Okres gwarancji producenta,
    Spalanie W Mieście,
    Numer rejestracyjny pojazdu,
    Leasing,
    Filtr cząstek stałych,
    Spalanie W Cyklu Mieszanym,
    Spalanie Poza Miastem,
    Uszkodzony,
    Gwarancja dealerska (w cenie),
    Miesięczna rata,
    PRIMARY KEY (ID)
    )'''
cur.execute(createTableCommand)
psql.commit()